In [1]:
!pip install numpy pandas scikit-learn tensorflow keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 33.8 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.0.5
    Uninstalling keras-3.0.5:
      Successfully uninstalled keras-3.0.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.


In [2]:
import numpy as np
import pandas as pd

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.utils import to_categorical

2024-03-18 18:12:13.954717: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-18 18:12:13.954885: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-18 18:12:14.115322: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
train = pd.read_csv("/kaggle/input/swahili-news-classification-zindi/Train.csv")
train.head(3)

,id,content,category
0,SW0,SERIKALI imesema haitakuwa tayari kuona amani...,Kitaifa
1,SW1,"Mkuu wa Mkoa wa Tabora, Aggrey Mwanri amesiti...",Biashara
2,SW10,SERIKALI imetoa miezi sita kwa taasisi zote z...,Kitaifa


In [4]:
test = pd.read_csv("/kaggle/input/swahili-news-classification-zindi/Test.csv")
test.head(3)

,swahili_id,content
0,001dd47ac202d9db6624a5ff734a5e7dddafeaf2,"MKUU wa Wilaya ya Bahi, Mkoani Dodoma, Mwanah..."
1,0043d97f7690e9bc02f0ed8bb2b260d1d44bad92,"MWISHONI mwa wiki hii, Timu ya Soka ya Taifa,..."
2,00579c2307b5c11003d21c40c3ecff5e922c3fd8,THAMANI ya mauzo ya bidhaa za Afrika Masharik...


In [5]:
text_data = train['content']
labels = train['category']

In [6]:
test_text = test['content']

In [7]:
tokenizer = Tokenizer(num_words =5000, oov_token='<OOV>')
tokenizer.fit_on_texts(text_data)
sequences = tokenizer.texts_to_sequences(text_data)

In [8]:
tokenizer2 = Tokenizer(num_words =5000, oov_token='<OOV>')
tokenizer2.fit_on_texts(test_text)
sequences2 = tokenizer2.texts_to_sequences(test_text)

In [9]:
max_sequence_length = 100
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding = 'post')

In [10]:
max_sequence_length = 100
test_sequences = pad_sequences(sequences2, maxlen=max_sequence_length, padding = 'post')

In [11]:
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
categorical_labels = to_categorical(encoded_labels)

In [12]:
X_train, X_valid, y_train, y_valid = train_test_split(padded_sequences, categorical_labels, test_size=0.2, random_state=42)

In [13]:
model = Sequential()
model.add(Embedding(5000, 128))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(len(set(labels)), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 128)         640000    
                                                                 
 spatial_dropout1d (Spatial  (None, None, 128)         0         
 Dropout1D)                                                      
                                                                 
 lstm (LSTM)                 (None, 100)               91600     
                                                                 
 dense (Dense)               (None, 5)                 505       
                                                                 
Total params: 732105 (2.79 MB)
Trainable params: 732105 (2.79 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [14]:
epochs = 10
batch_size = 64
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_valid, y_valid), verbose=1)

Epoch 1/10
65/65 [==============================] - 11s 126ms/step - loss: 1.1581 - accuracy: 0.4536 - val_loss: 1.2546 - val_accuracy: 0.4888
Epoch 2/10
65/65 [==============================] - 8s 117ms/step - loss: 0.7239 - accuracy: 0.6617 - val_loss: 0.6559 - val_accuracy: 0.7129
Epoch 3/10
65/65 [==============================] - 8s 119ms/step - loss: 0.5716 - accuracy: 0.7473 - val_loss: 0.5968 - val_accuracy: 0.7371
Epoch 4/10
65/65 [==============================] - 8s 119ms/step - loss: 0.4286 - accuracy: 0.8340 - val_loss: 0.5509 - val_accuracy: 0.7740
Epoch 5/10
65/65 [==============================] - 8s 123ms/step - loss: 0.3169 - accuracy: 0.8869 - val_loss: 0.5488 - val_accuracy: 0.7876
Epoch 6/10
65/65 [==============================] - 8s 116ms/step - loss: 0.2352 - accuracy: 0.9211 - val_loss: 0.6725 - val_accuracy: 0.7915
Epoch 7/10
65/65 [==============================] - 8s 119ms/step - loss: 0.1915 - accuracy: 0.9362 - val_loss: 0.6291 - val_accuracy: 0.7740
Epoch

In [15]:
loss, accuracy = model.evaluate(X_valid, y_valid, verbose=1)
print("Test Accuracy: {:.2f}%".format(accuracy * 100))

33/33 [==============================] - 1s 17ms/step - loss: 0.7978 - accuracy: 0.7886
Test Accuracy: 78.86%


In [16]:
preds = model.predict(test_sequences)
preds

41/41 [==============================] - 1s 18ms/step


array([[2.3965733e-01, 2.7533378e-02, 8.8230960e-02, 6.0132468e-01,
        4.3253671e-02],
       [1.7279314e-04, 5.0584506e-05, 3.4538083e-04, 1.3371801e-03,
        9.9809396e-01],
       [1.1797577e-01, 2.7026623e-03, 1.5900275e-02, 7.2221816e-01,
        1.4120314e-01],
       ...,
       [2.1303695e-01, 8.0287438e-03, 1.8289855e-02, 7.3303676e-01,
        2.7607627e-02],
       [2.7733645e-03, 4.6549609e-04, 2.8741371e-03, 6.8483591e-01,
        3.0905095e-01],
       [1.3217659e-02, 1.3534422e-03, 4.6315189e-03, 9.7613925e-01,
        4.6581211e-03]], dtype=float32)

In [17]:
predicted_labels = np.argmax(preds, axis=1)

predicted_categories = label_encoder.inverse_transform(predicted_labels)
predicted_categories

array(['Kitaifa', 'michezo', 'Kitaifa', ..., 'Kitaifa', 'Kitaifa',
       'Kitaifa'], dtype=object)

In [18]:
submission_df = pd.DataFrame({'ID': test['swahili_id'], 'category': predicted_categories})
submission_df.head(3)

,ID,category
0,001dd47ac202d9db6624a5ff734a5e7dddafeaf2,Kitaifa
1,0043d97f7690e9bc02f0ed8bb2b260d1d44bad92,michezo
2,00579c2307b5c11003d21c40c3ecff5e922c3fd8,Kitaifa


In [19]:
one_hot_encoded = pd.get_dummies(submission_df['category']).astype(int)
df_encoded = pd.concat([submission_df, one_hot_encoded], axis=1)
df_encoded.drop(columns=['category'], inplace=True)

df_encoded.head(3)

,ID,Biashara,Burudani,Kimataifa,Kitaifa,michezo
0,001dd47ac202d9db6624a5ff734a5e7dddafeaf2,0,0,0,1,0
1,0043d97f7690e9bc02f0ed8bb2b260d1d44bad92,0,0,0,0,1
2,00579c2307b5c11003d21c40c3ecff5e922c3fd8,0,0,0,1,0


In [20]:
df_encoded.to_csv('submission.csv', index=False)